In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

# Task description

To train a variable misuse detection model one needs to implement an NLP labeling model.

For example, for a funciton containing misuse
```
def _eq(l1, l2):\n    return (set(l1) == set(l1))
```
the misuse character span is (44, 46). To do this with NLP methods, code is tokenized, and labels for tokens are generated
```
[def, _, eq, (, l, 1, ",", l, 2, ):, \n, \t, return, (, set, (, l1, ), ==, set, (, l1, ), ), ]
[O  , O, O , O, O, O,  O , O, O, 0 , O , O ,    O  , O, O  , O, O , O, O , O  , O, M , O, O, O
```
The goal is to train an NLP model that predicts those labels correctly. In this project, BILUO labeling scheme is used.

# Goal

The goal of this project
1. Verify dataset, make sure that encoded batches are correct (misuse spans are correct). You can sample dataset and make sure that the number of errors is less than a certain threshold.
2. Train variable misuse detection model (with finetuning and without)
3. Verify [scoring function](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/entity/type_prediction.py#L71)
4. Conduct a series of experiments to identify performance
5. Analyze errors

# Why using this example?

Basic functionality, necessary for train an NLP labeler is
1. Loading data (implemented in this example)
2. Tokenization, preparing labels (implemented in [`PythonBatcher.prepare_sent`](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/batchers/PythonBatcher.py#L123))
3. Data encoding for using with ML models (implemented in [`PythonBatcher.create_batches_with_mask`](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/batchers/PythonBatcher.py#L206))
4. Batching (implemented in [`PythonBatcher.format_batch`](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/batchers/PythonBatcher.py#L256))
5. Model training (partially implemented in [`CodeBertModelTrainer2.train_model`](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/codebert/codebert_train.py#L148) and extended here)
6. Tensorboard tracking (implemented in `CodeBertModelTrainer2`)

# Install libraries

1. See [installation steps](https://github.com/VitalyRomanov/method-embedding#installing-python-libraries).

2. Install transformers
```bash
pip install transformers
```

In [2]:
import os
from os.path import join
from argparse import Namespace
from SourceCodeTools.nlp.codebert.codebert_train import HybridModelTrainer, test_step, train_step_finetune, CodeGPT2HybridModel, batch_to_torch
from SourceCodeTools.nlp.entity.type_prediction import scorer
from transformers import GPT2Tokenizer, GPT2Model

import json
import torch
from time import time
from copy import copy
from datetime import datetime
import hashlib
import tempfile
from collections import defaultdict
from pathlib import Path
from math import ceil
from typing import Dict, Optional, List, Union
from SourceCodeTools.nlp.tokenizers import _inject_tokenizer

from tqdm import tqdm
import numpy as np

import diskcache as dc


from SourceCodeTools.nlp.entity import fix_incorrect_tags
from SourceCodeTools.code.annotator_utils import adjust_offsets, biluo_tags_from_offsets

2022-07-29 04:44:42.342592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-29 04:44:42.342611: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Definitions

## Tokenizer

Current code works with many tokenizers. The most comparible format for storing labels is to store them as character spans. Character spans for labels are mapped to tokens with Spacy's `biluo_tags_from_offsets`. For this reason, we need to have instruments to make tokenizers compatible with Spacy format.

In [3]:
class AdapterDoc:
    """
    A simple wrapper for tokens that also stores additional data such as character span adjustment and 
    tokens compatible with `biluo_tags_from_offsets`
    """
    def __init__(self, tokens):
        self.tokens = tokens
        self.adjustment_amount = 0
        self.tokens_for_biluo_alignment = None

    def __iter__(self):
        return iter(self.tokens)

    def __repr__(self):
        return "".join(self.tokens)
    
    def __len__(self):
        return len(self.tokens)

class ModelAdapter:
    def __init__(self, primary_tokenization):
        self.primary_tokenization = primary_tokenization
        self.nlp = spacy.blank("en")
        self.regex_tok = create_tokenizer("regex")

    def secondary_tokenization(self, tokens):
        new_tokens = []
        for token in tokens:
            new_tokens.extend(self.regex_tok(token))
        return new_tokens

    def __call__(self, text):
        tokens = self.primary_tokenization(text)
        tokens = self.secondary_tokenization(tokens)
        doc = Doc(self.nlp.vocab, tokens, spaces=[False] * len(tokens))

        backup_tokens = doc
        fixed_spaces = [False]
        fixed_words = ["<s>"]

        for ind, t in enumerate(doc):
            if len(t.text) > 1:
                fixed_words.append(t.text.strip("Ġ"))
            else:
                fixed_words.append(t.text)
            if ind != 0:
                fixed_spaces.append(t.text.startswith("Ġ") and len(t.text) > 1)
        fixed_spaces.append(False)
        fixed_spaces.append(False)
        fixed_words.append("</s>")

        assert len(fixed_spaces) == len(fixed_words)

        doc = Doc(self.nlp.vocab, fixed_words, fixed_spaces)

        assert len(doc) - 2 == len(backup_tokens)
        assert len(doc.text) - 7 == len(backup_tokens.text)

        final_doc = AdapterDoc(["<s>"] + [t.text for t in backup_tokens] + ["</s>"])
        final_doc.adjustment_amount = -3
        final_doc.tokens_for_biluo_alignment = doc

        return final_doc
    
    
def create_tokenizer(type, bpe_path=None, regex=None):
    if type == "spacy":
        import spacy
        print("Creating spacy tokenizer")
        return _inject_tokenizer(spacy.blank("en"))
    elif type == "codebert":
        from transformers import RobertaTokenizer
        import spacy
        from spacy.tokens import Doc

        tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
        adapter = ModelAdapter(primary_tokenization=tokenizer.tokenize)

        def tokenize(text):
            return adapter(text)

        return tokenize
    elif type == "codegpt2":
        print('came here')
        from transformers import GPT2Tokenizer
        tokenizer = GPT2Tokenizer.from_pretrained("microsoft/CodeGPT-small-py")
        adapter = ModelAdapter(primary_tokenization=tokenizer.tokenize)

        def tokenize(text):
            return adapter(text)
        
        return tokenize
    else:
        raise Exception("Supported tokenizer types: spacy, regex, bpe")

## Batcher

In [4]:
from typing import Dict, Optional, List, Union
from SourceCodeTools.nlp import create_tokenizer, tag_map_from_sentences, TagMap, token_hasher, try_int, ValueEncoder


class SampleEntry(object):
    def __init__(self, id, text, labels=None, category=None, **kwargs):
        self._storage = dict()
        self._storage["id"] = id
        self._storage["text"] = text
        self._storage["labels"] = labels
        self._storage["category"] = category
        self._storage.update(kwargs)

    def __getattr__(self, item):
        storage = object.__getattribute__(self, "_storage")
        if item in storage:
            return storage[item]
        return super().__getattribute__(item)

    def __repr__(self):
        return repr(self._storage)

    def __getitem__(self, item):
        return self._storage[item]

    def keys(self):
        return list(self._storage.keys())


class MapperSpec:
    def __init__(self, field, target_field, encoder, dtype=np.int32, preproc_fn=None):
        self.field = field
        self.target_field = target_field
        self.encoder = encoder
        self.preproc_fn = preproc_fn
        self.dtype = dtype

        
class SimplePythonBatcher:
    def __init__(
            self, data, batch_size: int, seq_len: int,
            wordmap: Dict[str, int], *, tagmap: Optional[TagMap] = None,
            class_weights=False, element_hash_size=1000, sort_by_length=True, tokenizer="spacy", no_localization=False,
            cache_dir: Optional[Union[str, Path]] = None, **kwargs
    ):

        self._batch_size = batch_size
        self._max_seq_len = seq_len
        self._tokenizer = tokenizer
        self._class_weights = None
        self._no_localization = no_localization
        self._nlp = create_tokenizer(tokenizer)
        self._cache_dir = Path(cache_dir) if cache_dir is not None else cache_dir
        self._valid_sentences = 0
        self._filtered_sentences = 0
        self._wordmap = wordmap
        self.tagmap = tagmap
        self._sort_by_length = sort_by_length

        self._create_cache()
        self._prepare_data(data)
        self._create_mappers(**kwargs)

    def _get_version_code(self):
        defining_parameters = json.dumps({
            "tokenizer": self._tokenizer, "max_seq_len": self._max_seq_len
        })
        return self._compute_text_id(defining_parameters)

    @staticmethod
    def _compute_text_id(text):
        return int(hashlib.md5(text.encode('utf-8')).hexdigest(), 16) % 1152921504606846976

    def _check_cache_dir(self):
        if not hasattr(self, "_cache_dir") or self._cache_dir is None:
            raise Exception("Cache directory location has not been specified yet")

    def _get_cache_location_name(self, cache_name):
        self._check_cache_dir()
        return str(self._cache_dir.joinpath(cache_name))

    @property
    def _data_cache_path(self):
        return self._get_cache_location_name("DataCache")

    # @property
    # def _sent_cache_path(self):
    #     return self._get_cache_location_name("SentCache")

    @property
    def _batch_cache_path(self):
        return self._get_cache_location_name("BatchCache")

    @property
    def _tagmap_path(self):
        return self._cache_dir.joinpath("tagmap.json")

    def _create_cache(self):
        if self._cache_dir is None:
            self._tmp_dir = tempfile.TemporaryDirectory()
            self._cache_dir = Path(self._tmp_dir.name)

        self._cache_dir = self._cache_dir.joinpath(f"PythonBatcher{self._get_version_code()}")
        self._cache_dir.mkdir(parents=True, exist_ok=True)
        print(f"Created cache directory at location {self._cache_dir}")

        self._data_cache = dict()  #dc.Cache(self._data_cache_path)
        # self._sent_cache = dc.Cache(self._sent_cache_path)
        self._batch_cache = dc.Cache(self._batch_cache_path)

    def _prepare_data(self, data):
        self._sent_lenghts = {}

        for text, annotations in tqdm(data, desc="Preprocess functions"):
            id_ = self._compute_text_id(text)
            if id_ not in self._data_cache:
                extra = copy(annotations)
                labels = extra.pop("entities")
                extra.update(self._prepare_tokenized_sent((text, annotations)))
                entry = SampleEntry(id=id_, text=text, labels=labels, **extra)
                self._data_cache[id_] = entry
            else:
                entry = self._data_cache[id_]
            self._sent_lenghts[id_] = len(entry.tokens)
        
    def _iterate_record_ids(self):
        return self._data_cache.iterkeys()
    
    def _get_record_with_id(self, id):
        if id not in self._data_cache:
            raise KeyError("Record with such id is not found")
        return self._data_cache[id]

    def _iterate_sorted_by_length(self, limit_max_length=False):
        for id_, length in sorted(self._sent_lenghts.items(), key=lambda x: x[1]):
            if limit_max_length and length >= self._max_seq_len:
                continue
            yield self._get_record_with_id(id_)

    def _iterate_records(self, limit_max_length=False, shuffle=False):
        for id_ in self._sent_lenghts.keys():
            if limit_max_length and self._sent_lenghts[id_] >= self._max_seq_len:
                continue
            yield self._get_record_with_id(id_)

    def _create_mappers(self, **kwargs):
        self._mappers = []
        self._create_wordmap_encoder()
        self._create_tagmap_encoder()

    def _create_tagmap_encoder(self):
        if self.tagmap is None:
            if self._tagmap_path.is_file():
                tagmap = TagMap.load(self._tagmap_path)
            else:
                def iterate_tags():
                    for record in self._iterate_records():
                        for label in record.tags:
                            yield label

                tagmap = tag_map_from_sentences(iterate_tags())
                tagmap.set_default(tagmap._value_to_code["O"])
                tagmap.save(self._tagmap_path)

            self.tagmap = tagmap

        self._mappers.append(
            MapperSpec(field="tags", target_field="tags", encoder=self.tagmap)
        )
        # self.tagmap = tagmap
        # self.tagpad = self.tagmap["O"]

    def _create_wordmap_encoder(self):
        wordmap_enc = ValueEncoder(value_to_code=self._wordmap)
        wordmap_enc.set_default(len(self._wordmap))
        self._mappers.append(
            MapperSpec(field="tokens", target_field="tok_ids", encoder=wordmap_enc)
        )

    @property
    def num_classes(self):
        return len(self.tagmap)

    def _prepare_tokenized_sent(self, sent):
        text, annotations = sent

        doc = self._nlp(text)
        ents = annotations['entities']

        tokens = doc
        try:
            tokens = [t.text for t in tokens]
        except:
            pass

        if hasattr(doc, "tokens_for_biluo_alignment"):
            entity_adjustment_amount = doc.adjustment_amount
            tokens_for_biluo_alignment = doc.tokens_for_biluo_alignment
        else:
            entity_adjustment_amount = 0
            tokens_for_biluo_alignment = doc

        ents_tags = biluo_tags_from_offsets(
            tokens_for_biluo_alignment, adjust_offsets(ents, entity_adjustment_amount),
            self._no_localization
        )
        fix_incorrect_tags(ents_tags)

        assert len(tokens) == len(ents_tags)

        output = {
            "tokens": tokens,
            "tags": ents_tags
        }

        return output

    # @lru_cache(maxsize=200000)
    def _encode_for_batch(self, record):

        if record.id in self._batch_cache:
            return self._batch_cache[record.id]

        def encode(seq, encoder, pad, preproc_fn=None):
            if preproc_fn is None:
                def preproc_fn(x):
                    return x
            blank = np.ones((self._max_seq_len,), dtype=np.int32) * pad
            encoded = np.array([encoder[preproc_fn(w)] for w in seq], dtype=np.int32)
            blank[0:min(encoded.size, self._max_seq_len)] = encoded[0:min(encoded.size, self._max_seq_len)]
            return blank

        output = {}

        for mapper in self._mappers:
            output[mapper.target_field] = encode(
                seq=record[mapper.field], encoder=mapper.encoder, pad=mapper.encoder.default,
                preproc_fn=mapper.preproc_fn
            ).astype(mapper.dtype)

        tokens = record.tokens
        num_tokens = len(tokens)

        # assert len(s) == len(t)

        output["no_loc_mask"] = np.array([tag != self.tagmap.default for tag in output["tags"]]).astype(np.bool)
        output["lens"] = num_tokens if num_tokens < self._max_seq_len else self._max_seq_len

        self._batch_cache[record.id] = output

        return output

    def format_batch(self, batch):
        fbatch = defaultdict(list)

        for sent in batch:
            for key, val in sent.items():
                fbatch[key].append(val)

        max_len = max(fbatch["lens"])

        return {
            key: np.stack(val)[:,:max_len] if key != "lens" and key != "replacements" and key != "tokens"
            else (np.array(val, dtype=np.int32) if key == "lens" else np.array(val)) for key, val in fbatch.items()}

    def generate_batches(self):
        batch = []
        if self._sort_by_length:
            records = self._iterate_sorted_by_length(limit_max_length=True)
        else:
            records = self._iterate_records(limit_max_length=True, shuffle=False)

        for sent in records:
            batch.append(self._encode_for_batch(sent))
            if len(batch) >= self._batch_size:
                yield self.format_batch(batch)
                batch = []
        if len(batch) > 0:
            yield self.format_batch(batch)
        # yield self.format_batch(batch)

    def __iter__(self):
        return self.generate_batches()

    def __len__(self):
        total_valid = sum(1 for id_, length in self._sent_lenghts.items() if length < self._max_seq_len)
        return int(ceil(total_valid / self._batch_size))

## Reading data

In [5]:
def read_data(dataset_path, partition):
    """
    Read data storead as JSON records.
    """
    assert partition in {"train", "val", "test"}
    data_path = join(dataset_path, f"var_misuse_seq_{partition}.json")
    
    data = []
    for line in open(data_path, "r"):
        entry = json.loads(line)
        
        text = entry.pop("text")
        data.append((text, entry))
        
    return data

## Training procedure

In [6]:
class VariableMisuseDetector(HybridModelTrainer):
    def set_batcher_class(self):
        self.batcher = SimplePythonBatcher
        
    def get_batcher(self, *args, **kwargs):
        kwargs.update({"tokenizer": self.model_name})
        return self.batcher(*args, **kwargs)

    def get_trial_dir(self):
        """
        Define folder name format for storing checkpoints.
        """
        return os.path.join(self.output_dir, f"{self.model_name}_var_mususe" + str(datetime.now())).replace(":", "-").replace(" ", "_")
    
    def train(
            self, model, train_batches, test_batches, epochs, report_every=10, scorer=None, learning_rate=0.01,
            learning_rate_decay=1., finetune=False, summary_writer=None, save_ckpt_fn=None, no_localization=False
    ):
        # all training options are specified [here](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/entity/type_prediction.py#L256)

        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=learning_rate_decay)  # there is no learning rate decay by default

        # metric history is stored here
        train_losses = []
        test_losses = []
        train_f1s = []
        test_f1s = []

        num_train_batches = len(train_batches)
        num_test_batches = len(test_batches)

        best_f1 = 0.

        for e in range(epochs):
            losses = []
            ps = []
            rs = []
            f1s = []

            start = time()
            model.train()

            for ind, batch in enumerate(tqdm(train_batches)):
                batch_to_torch(batch, self.device)  # inspect the content of `batch`
                loss, p, r, f1 = train_step_finetune(
                    model=model, optimizer=optimizer, token_ids=batch['tok_ids'],
                    prefix=None, suffix=None, graph_ids=None,  # keep this None
                    labels=batch['tags'], lengths=batch['lens'],
                    extra_mask=None,  # Keep this None
                    scorer=scorer,
                    finetune=finetune and e / epochs > 0.2,  # finetuning starts after 20% of training is complete
                    vocab_mapping=self.vocab_mapping
                )
                losses.append(loss.cpu().item())
                ps.append(p)
                rs.append(r)
                f1s.append(f1)

                self.summary_writer.add_scalar("Loss/Train", loss, global_step=e * num_train_batches + ind)
                self.summary_writer.add_scalar("Precision/Train", p, global_step=e * num_train_batches + ind)
                self.summary_writer.add_scalar("Recall/Train", r, global_step=e * num_train_batches + ind)
                self.summary_writer.add_scalar("F1/Train", f1, global_step=e * num_train_batches + ind)

            test_alosses = []
            test_aps = []
            test_ars = []
            test_af1s = []

            model.eval()

            for ind, batch in enumerate(test_batches):
                batch_to_torch(batch, self.device)
                
                test_loss, test_p, test_r, test_f1 = test_step(
                    model=model, token_ids=batch['tok_ids'],
                    prefix=None, suffix=None, graph_ids=None,  # keep this None
                    labels=batch['tags'], lengths=batch['lens'],
                    extra_mask=None,  # keep this None
                    scorer=scorer, vocab_mapping=self.vocab_mapping
                )

                self.summary_writer.add_scalar("Loss/Test", test_loss, global_step=e * num_test_batches + ind)
                self.summary_writer.add_scalar("Precision/Test", test_p, global_step=e * num_test_batches + ind)
                self.summary_writer.add_scalar("Recall/Test", test_r, global_step=e * num_test_batches + ind)
                self.summary_writer.add_scalar("F1/Test", test_f1, global_step=e * num_test_batches + ind)
                test_alosses.append(test_loss.cpu().item())
                test_aps.append(test_p)
                test_ars.append(test_r)
                test_af1s.append(test_f1)

            epoch_time = time() - start

            train_losses.append(float(sum(losses) / len(losses)))
            train_f1s.append(float(sum(f1s) / len(f1s)))
            test_losses.append(float(sum(test_alosses) / len(test_alosses)))
            test_f1s.append(float(sum(test_af1s) / len(test_af1s)))

            print(
                f"Epoch: {e}, {epoch_time: .2f} s, Train Loss: {train_losses[-1]: .4f}, Train P: {sum(ps) / len(ps): .4f}, Train R: {sum(rs) / len(rs): .4f}, Train F1: {sum(f1s) / len(f1s): .4f}, "
                f"Test loss: {test_losses[-1]: .4f}, Test P: {sum(test_aps) / len(test_aps): .4f}, Test R: {sum(test_ars) / len(test_ars): .4f}, Test F1: {test_f1s[-1]: .4f}")

            if save_ckpt_fn is not None and float(test_f1s[-1]) > best_f1:
                save_ckpt_fn()
                best_f1 = float(test_f1s[-1])

            scheduler.step(epoch=e)

        return train_losses, train_f1s, test_losses, test_f1s
    
    def train_model(self, cache_dir=None):
        
        model_params = copy(self.model_params)

        print(f"\n\n{model_params}")
        lr = model_params.pop("learning_rate")
        lr_decay = model_params.pop("learning_rate_decay")
        suffix_prefix_buckets = model_params.pop("suffix_prefix_buckets")  # used for another model, ignore

        print("Creating dataloaders")
        train_batcher, test_batcher = self.get_dataloaders(word_emb=None, graph_emb=None, suffix_prefix_buckets=suffix_prefix_buckets, cache_dir=cache_dir)

        print("Loading pretrained model")
        
        print("Creating model")
        # definition of CodeGPT2Hybrid is at https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/codebert/codebert_train.py#L21
        model = self.hybrid_model_class(
            self.base_model, graph_emb=None, padding_idx=0, num_classes=train_batcher.num_classes,
            no_graph=self.no_graph
        )
                
        if self.use_cuda:
            model.cuda()

        trial_dir = self.get_trial_dir()  # create directory for saving checkpoints
        os.mkdir(trial_dir)
        self.create_summary_writer(trial_dir)
        
        train_batcher.tagmap.save(os.path.join(trial_dir, "tagmap.json"))
        # pickle.dump(train_batcher.tagmap, open(os.path.join(trial_dir, "tag_types.pkl"), "wb"))

        def save_ckpt_fn():
            checkpoint_path = os.path.join(trial_dir, "checkpoint")
            torch.save(model, open(checkpoint_path, 'wb'))

        print("Begin training")
        train_losses, train_f1, test_losses, test_f1 = self.train(
            model=model, train_batches=train_batcher, test_batches=test_batcher,
            epochs=self.epochs, learning_rate=lr,
            scorer=lambda pred, true: scorer(pred, true, train_batcher.tagmap, no_localization=self.no_localization),  # need to verify scoring function
            learning_rate_decay=lr_decay, finetune=self.finetune, save_ckpt_fn=save_ckpt_fn,
            no_localization=self.no_localization
        )

        metadata = {
            "train_losses": train_losses,
            "train_f1": train_f1,
            "test_losses": test_losses,
            "test_f1": test_f1,
            "learning_rate": lr,
            "learning_rate_decay": lr_decay,
            "epochs": self.epochs,
            "suffix_prefix_buckets": suffix_prefix_buckets,
            "seq_len": self.seq_len,
            "batch_size": self.batch_size,
            "no_localization": self.no_localization
        }

        print("Maximum f1:", max(test_f1))

        metadata.update(model_params)

        with open(os.path.join(trial_dir, "params.json"), "w") as metadata_sink:
            metadata_sink.write(json.dumps(metadata, indent=4))

# Execution

All training options are specified [here](https://github.com/VitalyRomanov/method-embedding/blob/e995477db13a13875cca54c37d4d29f63b0c8e93/SourceCodeTools/nlp/entity/type_prediction.py#L256)
Option names are added to `args` below.

In [7]:
dataset_path = "variable_misuse_graph_2_percent_balanced"

args = Namespace()
args.__dict__.update({
    "learning_rate": 1e-3,           #
    "max_seq_len": 512,              # default for BERT
    "random_seed": 42,               #
    "epochs": 10,                   #
    "gpu": -1,                       # set this to GPU id to use gpu
    "batch_size": 8,                 # higher value increases memory consumption
    "finetune": True,  # set this flag to enable finetuning
    "no_localization": False,        # whether to solve variable misuse with, or without localization
    
    # do not change items below
    "no_graph": True,                # used for another model
    "model_output": dataset_path,    # where to store checkpoints
    "graph_emb_path": None,          # used for another model
    "word_emb_path": None,           # used for another model
    "trials": 1,                     # setting > 1 repeats training, used to accumulate statisitcs
})

In [8]:
train_data = read_data(dataset_path, "train")
test_data = read_data(dataset_path, "val")

In [9]:
# test_data[0]  # ignore `replacements`

In [10]:
# test_data[100]

In [11]:
base_model = GPT2Model.from_pretrained("microsoft/CodeGPT-small-py")
base_model_tokenizer = GPT2Tokenizer.from_pretrained("microsoft/CodeGPT-small-py")
model_name = "codegpt2"
hybrid_model_class = CodeGPT2HybridModel

trainer = VariableMisuseDetector(
    hybrid_model_class, model_name, base_model_tokenizer, base_model, train_data, test_data,
    params={"learning_rate": 1e-4, "learning_rate_decay": 0.99, "suffix_prefix_buckets": 1},
    graph_emb_path=args.graph_emb_path, word_emb_path=args.word_emb_path,
    output_dir=args.model_output, epochs=args.epochs, batch_size=args.batch_size, gpu_id=args.gpu,
    finetune=args.finetune, trials=args.trials, seq_len=args.max_seq_len, no_localization=args.no_localization,
    no_graph=args.no_graph
)

Some weights of the model checkpoint at microsoft/CodeGPT-small-py were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/bl7awy/miniconda3/envs/SourceCodeTools/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.2 and is not supported. 
Some things might work, some things mi

In [12]:
trainer.train_model(cache_dir=Path(dataset_path).joinpath("__cache__"))



{'learning_rate': 0.0001, 'learning_rate_decay': 0.99, 'suffix_prefix_buckets': 1}
Creating dataloaders


Preprocess functions:   1%|                  | 44/7012 [00:00<00:15, 439.18it/s]

Created cache directory at location variable_misuse_graph_2_percent_balanced/__cache__/PythonBatcher220049262397976649


Preprocess functions:   7%|█▍                 | 54/732 [00:00<00:01, 538.94it/s]

Created cache directory at location variable_misuse_graph_2_percent_balanced/__cache__/PythonBatcher220049262397976649


  0%|                                           | 1/791 [00:00<02:14,  5.87it/s]

Loading pretrained model
Creating model
Begin training


 11%|████▋                                     | 88/791 [00:25<03:19,  3.52it/s]


KeyboardInterrupt: 

## Models to test

- CodeBert ([Huggingface](https://huggingface.co/microsoft/codebert-base))
- CodeGPT-2 ([Huggingface](https://huggingface.co/microsoft/CodeGPT-small-py))
- GraphCodeBert ([Huggingface](https://huggingface.co/microsoft/graphcodebert-base), [GitHub](https://github.com/microsoft/CodeBERT/tree/master/GraphCodeBERT/refinement))



## Visualizing results

Check example in the script

`SourceCodeTools/nlp/entity/utils/visualize_dataset.py`